In [1]:
%matplotlib qt 
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sqrt

In [2]:
alpha = 0.57
beta = 0.9
gamma = 2.32

Jcref = 3000  # A/mm²
Bc20 = 14.5
Tc0 = 9.2
C0 = 27.04


def calc_Jc(B,T):
    t = T/Tc0
    Bc2 = Bc20*(1-pow(t,1.7))
    B = np.clip(B,0,Bc2)
    b = B/Bc2
    
    return Jcref*C0 * pow(B,alpha-1) / (pow(Bc2,alpha)) * pow(1-b,beta) * pow(1-pow(t,1.7), gamma)

def init_critical_surface():
    B = np.arange(1,Bc20,1)
    B = np.insert(B,0,0.1)
    T = np.arange(0,Tc0,0.1)
    
    Tcs_data = {}
    
    for b in B:
        Jc = [calc_Jc(b,t) for t in T]
        Tcs_data[b] = {}
        Tcs_data[b]['Jc'] = Jc
        Tcs_data[b]['T'] = T
        
    return Tcs_data

def calc_current_sharing_temp(Jc,B):
    tcs = init_critical_surface()
    return np.interp(B, list(tcs.keys()),
                     [np.interp(Jc, np.flip(tcs[b]['Jc']), np.flip(tcs[b]['T'])) for b in tcs.keys()])

In [3]:
Iop = 300
Bop = 1

B = Iop * Bop/228
Jc = 792.0*Iop/228

print(B,Jc)

Tcs = calc_current_sharing_temp(Jc,B)
Tjoule = (Tcs+Tc0)/2

print(Tcs,Tjoule)

1.3157894736842106 1042.1052631578948
7.6936507301469 8.44682536507345


In [4]:
from mpl_toolkits import mplot3d

B = np.arange(1,Bc20,1)
B = np.insert(B,0,0.5)
T = np.arange(0,Tc0,0.5)
Bm,Tm = np.meshgrid(B,T)
Jc = calc_Jc(Bm,Tm)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(Bm, Tm, Jc/1000, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
ax.set_xlabel('Field [T]')
ax.set_ylabel('Temperature [K]')
ax.set_zlabel('Critical current density [kA/mm²]')



Text(0.5, 0, 'Critical current density [kA/mm²]')

In [69]:
B = np.arange(1.0,7.0,1)
B = np.insert(B,0,0.1)
T = np.arange(0,9.2,0.1)

ratio = 0.92
d_total = 0.856
s_total = pi*(d_total/2)**2
J_228 = 228*(ratio+1)/s_total
J_300 = 300*(ratio+1)/s_total

fig = plt.figure()
for b in B:
    Jcs = [calc_Jc(b,t) for t in T]
    plt.plot(T,Jcs, label= 'B = {} T'.format(b) )

plt.plot([0,10],[J_228,J_228],'--', label='Iop = 228 A')
plt.plot([0,10],[J_300,J_300],'--', label='Iop = 300 A')
plt.grid()
plt.xlabel('Temperature [K]')
plt.legend()
plt.ylabel('Critical current density [A/mm²]')
plt.ylim(0,5000)
plt.show()



In [ ]:
B = np.arange(1,15,0.1)
B = np.insert(B,0,0.5)
T = np.arange(4,7,1)

fig = plt.figure()

for t in T:
    Jcs = [calc_Jc(b,t) for b in B]
    plt.plot(B,Jcs, label= 'T = {} K'.format(t) )
    
plt.grid()
plt.xlabel('Field [T]')
plt.legend()
plt.ylabel('Critical current density [A/mm²]')
plt.ylim(0,5000)
plt.show()